# Complete BYOK Example App

> A full example FastHTML application demonstrating the BYOK system

In [1]:
#| default_exp examples.complete_app

## Setup and Imports

In [2]:
import os
from datetime import timedelta
from fasthtml.common import *
from fasthtml.jupyter import *

# BYOK Core imports
from cjm_fasthtml_byok.core.storage import BYOKManager
from cjm_fasthtml_byok.core.types import BYOKConfig, StorageBackend
from cjm_fasthtml_byok.middleware.beforeware import create_byok_beforeware, require_api_key
from cjm_fasthtml_byok.utils.helpers import get_key_summary, import_from_env, format_provider_name

# BYOK Component imports
from cjm_fasthtml_byok.components.forms import (
    KeyInputForm, 
    MultiProviderKeyForm,
    KeyManagementCard,
    KeyManagerDashboard,
    InlineKeyInput
)
from cjm_fasthtml_byok.components.alerts import (
    Alert,
    SecurityAlert,
    KeyStatusNotification,
    ValidationMessage,
    ToastContainer
)

# UI library imports
from cjm_fasthtml_daisyui.core.resources import get_daisyui_headers
from cjm_fasthtml_daisyui.components.navigation.navbar import navbar, navbar_start, navbar_center, navbar_end
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap, flex_display, flex_direction
from cjm_fasthtml_tailwind.utilities.spacing import p as padding, m as margin
from cjm_fasthtml_tailwind.utilities.sizing import w, max_w
from cjm_fasthtml_tailwind.core.base import combine_classes

## Create the FastHTML Application

In [3]:
# Configuration
SECRET_KEY = os.environ.get("SECRET_KEY", "dev-secret-key-change-in-production")
DATABASE_URL = "sqlite:///byok_demo.db"  # Use SQLite file for demo

# Configure BYOK
byok_config = BYOKConfig(
    storage_backend=StorageBackend.HYBRID,  # Use both session and database
    default_ttl=timedelta(hours=24),
    auto_cleanup=True,
    require_https=False  # Disabled for demo
)

# Initialize BYOK manager with SQLAlchemy
byok = BYOKManager(
    secret_key=SECRET_KEY,
    db_url=DATABASE_URL,  # Pass database URL instead of db object
    config=byok_config
)

# Setup BYOK beforeware
def get_user_id(req):
    """Get user ID from session (mock for demo)"""
    # Access session from req.session, not req.scope['session']
    if hasattr(req, 'session'):
        return req.session.get("user_id", "demo-user")
    return "demo-user"

# Create the beforeware handler
byok_beforeware = create_byok_beforeware(byok)

# Create the FastHTML app with session support and beforeware
from starlette.middleware.sessions import SessionMiddleware

app, rt = fast_app(
    hdrs=get_daisyui_headers(include_themes=True),
    secret_key=SECRET_KEY,
    sess_cls=SessionMiddleware,  # Use proper SessionMiddleware
    before=byok_beforeware,  # Add the beforeware here
    live=False
)

## Define Layout Components

In [4]:
def AppNavbar(req):
    """Create the application navbar"""
    return Div(
        Div(
            A("🔑 BYOK Demo", href="/", cls="btn btn-ghost text-xl"),
            cls=str(navbar_start)
        ),
        Div(
            A("Dashboard", href="/", cls="btn btn-ghost"),
            A("Add Key", href="/add", cls="btn btn-ghost"),
            A("Settings", href="/settings", cls="btn btn-ghost"),
            cls=str(navbar_end)
        ),
        cls=str(navbar)
    )

def PageLayout(req, title: str, *content):
    """Standard page layout"""
    return Titled(
        title,
        AppNavbar(req),
        Div(
            *content,
            cls=combine_classes("container", "mx-auto", padding._4, max_w.screen_xl)
        ),
        ToastContainer(position="top", align="end")
    )

## Route Handlers

In [5]:
@rt("/")
def index(req, sess):
    """Main dashboard page"""
    # Set user ID in session
    sess["user_id"] = "demo-user"
    
    # Get stored keys summary
    providers = ["openai", "anthropic", "google", "groq", "fireworks", "xai"]
    user_id = get_user_id(req)
    
    # Check for any notifications
    notifications = []
    if msg := req.query_params.get("msg"):
        msg_type = req.query_params.get("type", "info")
        notifications.append(Alert(msg, kind=msg_type, dismissible=True))
    
    return PageLayout(
        req,
        "API Key Dashboard",
        Div(*notifications) if notifications else None,
        H2("Manage Your API Keys", cls="text-2xl font-bold mb-6"),
        KeyManagerDashboard(
            req,
            providers=providers,
            byok_manager=byok,
            user_id=user_id,
            base_url="/api/keys"
        )
    )

In [6]:
@rt("/add")
def add_key_page(req, sess):
    """Page to add a new API key"""
    providers = ["openai", "anthropic", "google", "groq", "fireworks", "xai"]
    
    return PageLayout(
        req,
        "Add API Key",
        Div(
            H2("Add New API Key", cls="text-2xl font-bold mb-6"),
            Div(
                MultiProviderKeyForm(
                    providers=providers,
                    action="/api/keys/add"
                ),
                cls=combine_classes(max_w.md, "mx-auto")
            )
        )
    )

In [7]:
@rt("/settings")
def settings_page(req, sess):
    """Settings page"""
    user_id = get_user_id(req)
    summary = get_key_summary(byok, req, user_id)
    
    return PageLayout(
        req,
        "Settings",
        H2("Settings", cls="text-2xl font-bold mb-6"),
        
        # Security status
        Div(
            H3("Security Status", cls="text-xl font-semibold mb-4"),
            SecurityAlert(
                "HTTPS is not enabled. API keys may be transmitted insecurely.",
                severity="medium" if req.url.scheme == "http" else "low"
            ) if req.url.scheme == "http" else Alert(
                "Connection is secure (HTTPS)",
                kind="success"
            ),
            cls="mb-8"
        ),
        
        # Key summary
        Div(
            H3("Stored Keys Summary", cls="text-xl font-semibold mb-4"),
            P(f"Total keys stored: {summary['count']}", cls="mb-2"),
            Ul(
                *[
                    Li(
                        f"{key['display_name']}: {key['masked_key']} - {key['created']}"
                    )
                    for key in summary['keys']
                ],
                cls="list-disc list-inside"
            ) if summary['keys'] else P("No keys stored", cls="text-gray-500"),
            cls="mb-8"
        ),
        
        # Actions
        Div(
            H3("Actions", cls="text-xl font-semibold mb-4"),
            Form(
                Button(
                    "Clear All Keys",
                    type="submit",
                    cls=combine_classes(btn, btn_colors.error),
                    onclick="return confirm('Are you sure you want to delete all keys?');"
                ),
                method="post",
                action="/api/keys/clear-all"
            )
        )
    )

## API Endpoints

In [8]:
@rt("/api/keys/add", methods=["POST"])
def add_key(req, sess, provider: str, api_key: str):
    """Add a new API key"""
    user_id = get_user_id(req)
    
    try:
        # Store the key
        byok.set_key(
            req,
            provider=provider,
            api_key=api_key,
            user_id=user_id,
            ttl=timedelta(days=30)
        )
        
        return RedirectResponse(
            url="/?msg=API+key+added+successfully&type=success",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+add+key:+{str(e)}&type=error",
            status_code=303
        )

In [9]:
@rt("/api/keys/{provider}", methods=["POST"])
def update_key(req, sess, provider: str, api_key: str):
    """Update an API key for a specific provider"""
    user_id = get_user_id(req)
    
    try:
        byok.set_key(
            req,
            provider=provider,
            api_key=api_key,
            user_id=user_id,
            ttl=timedelta(days=30)
        )
        
        return RedirectResponse(
            url=f"/?msg={provider}+key+updated&type=success",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+update+key:+{str(e)}&type=error",
            status_code=303
        )

In [10]:
@rt("/api/keys/{provider}/delete", methods=["POST"])
def delete_key(req, sess, provider: str):
    """Delete an API key"""
    user_id = get_user_id(req)
    
    try:
        byok.delete_key(req, provider, user_id)
        
        return RedirectResponse(
            url=f"/?msg={provider}+key+deleted&type=info",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+delete+key:+{str(e)}&type=error",
            status_code=303
        )

In [11]:
@rt("/api/keys/clear-all", methods=["POST"])
def clear_all_keys(req, sess):
    """Clear all stored API keys"""
    user_id = get_user_id(req)
    
    try:
        byok.clear_keys(req, user_id)
        
        return RedirectResponse(
            url="/?msg=All+keys+cleared&type=info",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+clear+keys:+{str(e)}&type=error",
            status_code=303
        )

## Protected Routes Example

In [12]:
@rt("/demo/openai")
@require_api_key("openai", user_id_func=get_user_id)
def openai_demo(req, sess):
    """Demo route that requires OpenAI API key"""
    user_id = get_user_id(req)
    # Get the API key using the global byok manager
    api_key = byok.get_key(req, "openai", user_id)
    
    return PageLayout(
        req,
        "OpenAI Demo",
        H2("OpenAI Integration", cls="text-2xl font-bold mb-4"),
        Alert(
            f"Successfully accessed with API key: {api_key[:10]}...",
            kind="success"
        ),
        P("This page requires an OpenAI API key to access.", cls="mt-4"),
        P("In a real application, you would use this key to make API calls to OpenAI.", cls="mt-2")
    )

## Run the Application

In [13]:
#| eval: false
# For Jupyter notebook display
from fasthtml.jupyter import *
from cjm_fasthtml_daisyui.core.testing import start_test_server

# Start the server
server = start_test_server(app, port=5001)

# Display in Jupyter
display(HTMX(port=server.port))

print("🚀 BYOK Demo App is running!")
print(f"📍 Visit http://localhost:{server.port} to see the app")
print("\nTest flow:")
print("1. Visit the dashboard - initially no keys")
print("2. Click 'Add Key' to add a new API key")
print("3. Enter a test key (e.g., 'sk-test123' for OpenAI)")
print("4. See the key appear in the dashboard")
print("5. Try updating or deleting keys")
print("6. Visit /demo/openai to see protected route (requires OpenAI key)")
print("7. Check Settings page for summary and security status")
print("\nPress Stop button in Jupyter to stop the server")

🚀 BYOK Demo App is running!
📍 Visit http://localhost:5001 to see the app

Test flow:
1. Visit the dashboard - initially no keys
2. Click 'Add Key' to add a new API key
3. Enter a test key (e.g., 'sk-test123' for OpenAI)
4. See the key appear in the dashboard
5. Try updating or deleting keys
6. Visit /demo/openai to see protected route (requires OpenAI key)
7. Check Settings page for summary and security status

Press Stop button in Jupyter to stop the server


In [14]:
#| eval: false
# Stop the server when done
server.stop()

## Alternative: Run as Standalone Script

In [15]:
#| eval: false
# To run as a standalone script, uncomment the following:
# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=5001)

## Testing Different Scenarios

### Test Scenarios to Try:

1. **Add Keys**: Add different API keys for various providers
2. **Update Keys**: Change existing keys
3. **Delete Keys**: Remove individual keys
4. **Protected Routes**: Try accessing `/demo/openai` with and without a key
5. **Security Alerts**: Check the security warnings on the Settings page
6. **Clear All**: Use the "Clear All Keys" button in Settings
7. **Session Persistence**: Keys should persist during the session
8. **Validation**: Try submitting empty forms

### Key Features Demonstrated:

- ✅ Secure key storage with encryption
- ✅ Session and database hybrid storage
- ✅ Multi-provider support
- ✅ Protected routes with `@require_api_key` decorator
- ✅ Visual feedback with alerts and notifications
- ✅ Responsive dashboard layout
- ✅ Security warnings and status
- ✅ Key management (CRUD operations)
- ✅ Clean UI with daisyUI and Tailwind CSS